In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import altair as alt

from functools import reduce

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/sample_submission.csv
/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/engine_metadata.csv
/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/site_metadata.csv
/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/dataset/dataset/data_141.csv
/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/dataset/dataset/data_202.csv
/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/dataset/dataset/data_26.csv
/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/dataset/dataset/data_165.csv
/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/dataset/dataset/data_124.csv
/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/dataset/dataset/data_178.csv
/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/dataset/dataset/data_33.csv
/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/dataset/dataset/data_49.csv
/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/dataset/dataset/data_79.csv
/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/dataset/dataset/data

In [2]:
def loading_df(df):
    return pd.read_csv(f"/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/dataset/dataset/{df}")

In [3]:
engine_metadata = pd.read_csv("/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/engine_metadata.csv")
site_metadata = pd.read_csv("/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/site_metadata.csv")

In [4]:
new = engine_metadata.merge(site_metadata, on=["CUSTOMER_NAME", "PLANT_NAME"])
new

,CUSTOMER_NAME,PLANT_NAME,ENGINE_ID,FILE_ID,LATITUDE,LONGITUDE,ELEVATION,FUEL_LHV
0,SPIFFY,SPIRITUAL-POLECAT,ENGINE_1,data_1.csv,61.170356,42.874767,112.000000,47331.28920
1,SPIFFY,SPIRITUAL-POLECAT,ENGINE_2,data_2.csv,61.170356,42.874767,112.000000,47331.28920
2,SPIFFY,SPIRITUAL-POLECAT,ENGINE_3,data_3.csv,61.170356,42.874767,112.000000,47331.28920
3,NONCHALANT,NIFTY-ROOK,ENGINE_1,data_4.csv,37.554516,49.908217,-29.000000,47358.23582
4,NONCHALANT,NIFTY-ROOK,ENGINE_2,data_5.csv,37.554516,49.908217,-29.000000,47358.23582
...,...,...,...,...,...,...,...,...
207,SCEPTICAL,GIGANTIC-EARWIG,ENGINE_2,data_209.csv,41.631139,55.981426,168.878861,47289.03400
208,MERCIFUL,PREMIUM-MOTH,ENGINE_1,data_210.csv,-26.175581,16.115096,1014.342346,47343.12242
209,MERCIFUL,PREMIUM-MOTH,ENGINE_2,data_211.csv,-26.175581,16.115096,1014.342346,47343.12242
210,MERCIFUL,PREMIUM-MOTH,ENGINE_3,data_212.csv,-26.175581,16.115096,1014.342346,47343.12242


In [5]:
def thermal_e_colum(file):
    data = loading_df(file)
    fuel_lvh = new.loc[new["FILE_ID"] == file]["FUEL_LHV"]
    fuel_column = []
    porcentage = []
    for i in range(8761):
        fuel_column.append(float(fuel_lvh))
        porcentage.append(100)

    data["fuel_column"] = fuel_column
    
    data["porcentage"] = porcentage
    
    data["THERMAL_e"] = (data["POWER"] / (data["FUEL_FLOW"] * data["fuel_column"])) * data["porcentage"]
    
    data.drop(["fuel_column", "porcentage"], axis=1 , inplace=True)
    
    return data

In [6]:
thermal = thermal_e_colum("data_1.csv")

In [7]:
thermal

,Unnamed: 0,CMP_SPEED,POWER,FUEL_FLOW,CO2,THERMAL_e
0,01/01/2021 00:00,7093.322089,9187.921323,0.663525,1.739632,29.255776
1,01/01/2021 01:00,7094.569052,9186.903547,0.663176,1.738718,29.267912
2,01/01/2021 02:00,7089.672026,9191.390126,0.663971,1.740801,29.247163
3,01/01/2021 03:00,7092.412620,9211.980770,0.665044,1.743615,29.265387
4,01/01/2021 04:00,7088.579389,9163.370698,0.662288,1.736389,29.232097
...,...,...,...,...,...,...
8756,31/12/2021 20:00,0.000000,NaN,NaN,NaN,NaN
8757,31/12/2021 21:00,0.000000,NaN,NaN,NaN,NaN
8758,31/12/2021 22:00,0.000000,NaN,NaN,NaN,NaN
8759,31/12/2021 23:00,0.000000,NaN,NaN,NaN,NaN


In [8]:
def sum_df(*args):
    dataframes = []
    for arg in args:
        csv_file = pd.read_csv(f"/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/dataset/dataset/{arg}", index_col=0)
   #     csv_file["Dates"] = pd.to_datetime(csv_file["Dates"])
        csv_file.index.name = "Date"
        csv_file = csv_file.fillna(0)
        csv_file.index = pd.to_datetime(csv_file.index)
        fuel_lvh = new.loc[new["FILE_ID"] == arg]["FUEL_LHV"]
        fuel_column = []
        porcentage = []
        for i in range(8761):
            fuel_column.append(float(fuel_lvh))
            porcentage.append(100)

        csv_file["fuel_column"] = fuel_column
    
        csv_file["porcentage"] = porcentage
    
        csv_file["THERMAL_e"] = (csv_file["POWER"] / (csv_file["FUEL_FLOW"] * csv_file["fuel_column"])) * csv_file["porcentage"]
    
        csv_file.drop(["fuel_column", "porcentage"], axis=1 , inplace=True)
        dataframes.append(
            csv_file.fillna(0)
        )
#    return dataframes
    thermal_e = []
    for df in dataframes:
        thermal_e.append(float(df["THERMAL_e"].mean()))
    efficiency = sum(thermal_e)/len(thermal_e)
    return reduce(lambda x, y: x.add(y, fill_value=0), dataframes), efficiency

In [9]:
def multitable_dataframe(df, group, selected_group):
    """Return the sum of the selected tables
    """
    select = df.groupby(group)["FILE_ID"].apply(list)[selected_group]
    return sum_df(*select)

In [10]:
engine1, efficiency1 = multitable_dataframe(engine_metadata, "ENGINE_ID", "ENGINE_1")
engine2, efficiency2 = multitable_dataframe(engine_metadata, "ENGINE_ID", "ENGINE_2")
engine3, efficiency3 = multitable_dataframe(engine_metadata, "ENGINE_ID", "ENGINE_3")
engine4, efficiency4 = multitable_dataframe(engine_metadata, "ENGINE_ID", "ENGINE_4")
engine5, efficiency5 = multitable_dataframe(engine_metadata, "ENGINE_ID", "ENGINE_5")
engine6, efficiency6 = multitable_dataframe(engine_metadata, "ENGINE_ID", "ENGINE_6")
engine7, efficiency7 = multitable_dataframe(engine_metadata, "ENGINE_ID", "ENGINE_7")
engine8, efficiency8 = multitable_dataframe(engine_metadata, "ENGINE_ID", "ENGINE_8")

In [11]:
source = pd.DataFrame({"Engine_type": [
            "Engine_1",
            "Engine_2",
            "Engine_3",
            "Engine_4",
            "Engine_5",
            "Engine_6",
            "Engine_7",
            "Engine_8"
        ],
        "value": [
            efficiency1,
            efficiency2,
            efficiency3,
            efficiency4,
            efficiency5,
            efficiency6,
            efficiency7,
            efficiency8
        ]})

In [12]:
source

,Engine_type,value
0,Engine_1,22.289093
1,Engine_2,22.346286
2,Engine_3,23.168484
3,Engine_4,22.743465
4,Engine_5,23.043314
5,Engine_6,21.137872
6,Engine_7,20.557539
7,Engine_8,21.638046


In [13]:
alt.Chart(source).mark_arc().encode(
    theta=alt.Theta(field="value", type="quantitative"),
    color=alt.Color(field="Engine_type", type="nominal"),
)

alt.Chart(...)